In [1]:
import os
os.chdir(os.path.expanduser(".."))

In [2]:
from src.collect.collect import SDMXCollector
import pandas as pd
from io import StringIO

In [3]:
collector = SDMXCollector("sdmx.oecd.org/public", 
                          "rest", 
                          {"startPeriod": "2023-Q3", "format": "csv"})

arg_list = [SDMXCollector.OECD_ALL_COUNTRIES, "", "", "", "", "Q", "XDC", "Y"]
flow_ref = ["OECD.SDD.TPS", "DSD_BOP@DF_BOP", ""]

sample = collector.get(flow_ref, arg_list)

In [4]:
pd.read_csv(StringIO(sample))

,DATAFLOW,REF_AREA,COUNTERPART_AREA,MEASURE,ACCOUNTING_ENTRY,FS_ENTRY,FREQ,UNIT_MEASURE,ADJUSTMENT,TIME_PERIOD,OBS_VALUE,OBS_STATUS,UNIT_MULT,CURRENCY,DECIMALS
0,OECD.SDD.TPS:DSD_BOP@DF_BOP(1.0),ZAF,WXD,CA,D,T,Q,XDC,Y,2023-Q3,659603.10,A,6,ZAR,2
1,OECD.SDD.TPS:DSD_BOP@DF_BOP(1.0),ZAF,WXD,CA,D,T,Q,XDC,Y,2023-Q4,686585.40,A,6,ZAR,2
2,OECD.SDD.TPS:DSD_BOP@DF_BOP(1.0),AUS,WXD,CA,B,T,Q,XDC,Y,2024-Q2,-10725.00,A,6,AUD,2
3,OECD.SDD.TPS:DSD_BOP@DF_BOP(1.0),AUS,WXD,G,D,T,Q,XDC,Y,2023-Q3,109593.00,A,6,AUD,2
4,OECD.SDD.TPS:DSD_BOP@DF_BOP(1.0),AUS,WXD,G,D,T,Q,XDC,Y,2023-Q4,107469.00,A,6,AUD,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,OECD.SDD.TPS:DSD_BOP@DF_BOP(1.0),ZAF,WXD,IN2,D,T,Q,XDC,Y,2023-Q3,31461.70,A,6,ZAR,2
2204,OECD.SDD.TPS:DSD_BOP@DF_BOP(1.0),ZAF,WXD,IN2,D,T,Q,XDC,Y,2023-Q4,31389.01,A,6,ZAR,2
2205,OECD.SDD.TPS:DSD_BOP@DF_BOP(1.0),ZAF,WXD,IN2,B,T,Q,XDC,Y,2023-Q3,-11252.84,A,6,ZAR,2
2206,OECD.SDD.TPS:DSD_BOP@DF_BOP(1.0),ZAF,WXD,IN2,B,T,Q,XDC,Y,2023-Q4,-12626.86,A,6,ZAR,2


In [11]:
arg_list = ["AUT", "M", "IRLT", "", "", "", ""]
flow_ref = ["OECD.SDD.STES", "DSD_KEI@DF_KEI", "4.0"]

sample = collector.get(flow_ref, arg_list)

In [12]:
pd.read_csv(StringIO(sample))

,DATAFLOW,REF_AREA,FREQ,MEASURE,UNIT_MEASURE,ACTIVITY,ADJUSTMENT,TRANSFORMATION,TIME_PERIOD,OBS_VALUE,OBS_STATUS,UNIT_MULT,DECIMALS,BASE_PER
0,OECD.SDD.STES:DSD_KEI@DF_KEI(4.0),AUT,M,IRLT,PA,_Z,_Z,_Z,2023-09,3.2942,A,0,1,NaN
1,OECD.SDD.STES:DSD_KEI@DF_KEI(4.0),AUT,M,IRLT,PA,_Z,_Z,_Z,2023-08,3.1801,A,0,1,NaN
2,OECD.SDD.STES:DSD_KEI@DF_KEI(4.0),AUT,M,IRLT,PA,_Z,_Z,_Z,2023-07,3.1249,A,0,1,NaN
3,OECD.SDD.STES:DSD_KEI@DF_KEI(4.0),AUT,M,IRLT,PA,_Z,_Z,_Z,2024-10,2.7438,A,0,1,NaN
4,OECD.SDD.STES:DSD_KEI@DF_KEI(4.0),AUT,M,IRLT,PA,_Z,_Z,_Z,2024-09,2.6895,A,0,1,NaN
5,OECD.SDD.STES:DSD_KEI@DF_KEI(4.0),AUT,M,IRLT,PA,_Z,_Z,_Z,2024-08,2.7626,A,0,1,NaN
6,OECD.SDD.STES:DSD_KEI@DF_KEI(4.0),AUT,M,IRLT,PA,_Z,_Z,_Z,2024-07,2.9776,A,0,1,NaN
7,OECD.SDD.STES:DSD_KEI@DF_KEI(4.0),AUT,M,IRLT,PA,_Z,_Z,_Z,2024-06,3.0384,A,0,1,NaN
8,OECD.SDD.STES:DSD_KEI@DF_KEI(4.0),AUT,M,IRLT,PA,_Z,_Z,_Z,2024-05,3.0161,A,0,1,NaN
9,OECD.SDD.STES:DSD_KEI@DF_KEI(4.0),AUT,M,IRLT,PA,_Z,_Z,_Z,2024-04,2.9861,A,0,1,NaN
